# Consulta de Endereço de Supermercados de Porto Alegre em áreas afetadas pelas enchentes

### O objetivo é conseguir as coordenadas geográficas dos registros de supermercados e levantar a lista de supermercados impactados na região para fins didáticos

In [1]:
# Importando bibliotecas

import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderQuotaExceeded, GeocoderUnavailable
from geopy.extra.rate_limiter import RateLimiter
from pykml import parser
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
import time

## Pegando coordenadas (latitude, longitude) a partir de um Endereço

### Dados retirados desse site para fins didáticos https://www.tiendeo.com.br/Lojas/porto-alegre/supermercados#google_vignette

In [2]:
# Importando arquivo XLSX com a lista de endereços
tabela_supermercados = pd.read_excel("Supermercados Porto Alegre.xlsx")

In [3]:
tabela_supermercados.head()

,lista_enderecos,latitude,longitude,afetado
0,Avenida Plínio Brasil Milano 2343. - Porto Ale...,NaN,NaN,NaN
1,"Av. A. J. Renner, 1603. - Porto Alegre, RS",NaN,NaN,NaN
2,"Avenida Diario de Noticias, 500. 90810-080 - P...",NaN,NaN,NaN
3,"Avenida Diario de Noticias, 500 Cristal Bairro...",NaN,NaN,NaN
4,"Av Aureliano Figueredo Pinto, 789 Cidade Baixa...",NaN,NaN,NaN


In [4]:
# Definindo configurações do geolocator para acessar API de coordenadas
geolocator = Nominatim(user_agent="COLOQUE AQUI UM EMAIL VÁLIDO", timeout = 10)

In [5]:
# Função para obter coordenadas através do endereço
def get_coordinates(address, attempt=1, max_attempts=3):
    try:
        # Buscando coordenadas pela API do GeoPY
        location = geolocator.geocode(address)
        return (location.latitude, location.longitude) if location else (None, None)
    except (GeocoderTimedOut, GeocoderQuotaExceeded, GeocoderUnavailable) as e:
        if attempt < max_attempts:
            time.sleep(2 ** attempt)  # Backoff exponencial
            return get_coordinates(address, attempt + 1, max_attempts)
        else:
            print(f"Geocoding falhou após {max_attempts} tentativas: {e}")
            return (None, None)
    except Exception as e:
        print(f"Erro ao geocodificar {address}: {e}")
        return (None, None)


In [6]:
# Iterar sobre cada registro da lista para buscar e registrar coordenadas

i = len(tabela_supermercados)

for index, row in tabela_supermercados.iterrows():
    # Passe o endereço da forma como estiver nos seus dados
    # Exemplo caso tenha cada item em uma coluna: 
    # address_str = f"{row['street']}, {row['number']}, {row['neighborhood']}, {row['city']}, {row['state']}, Brazil" 
    address_str = row['lista_enderecos']
    lat, lon = get_coordinates(address_str)
    # Insere os registros na tabela
    tabela_supermercados.at[index, 'latitude'] = lat
    tabela_supermercados.at[index, 'longitude'] = lon
    
    print(i)
    # Decresce o contador de iterações
    i -= 1
    print(f"Endereço: {address_str}\nLatitude: {lat}, Longitude: {lon}\n")

    time.sleep(1)  # Para evitar excesso de solicitações à API

112
Endereço: Avenida Plínio Brasil Milano 2343. - Porto Alegre, RS
Latitude: -30.0132295, Longitude: -51.17090174375667

111
Endereço: Av. A. J. Renner, 1603. - Porto Alegre, RS
Latitude: -29.9829553, Longitude: -51.1921047

110
Endereço: Avenida Diario de Noticias, 500. 90810-080 - Porto Alegre, RS
Latitude: -30.0832343, Longitude: -51.2446005

109
Endereço: Avenida Diario de Noticias, 500 Cristal Bairro. 90810-080 - Porto Alegre, RS
Latitude: None, Longitude: None

108
Endereço: Av Aureliano Figueredo Pinto, 789 Cidade Baixa Bairro. 90050-191 - Porto Alegre, RS
Latitude: None, Longitude: None

107
Endereço: Rua Fernando Machado, 860 – Centro. 90010-320 - Porto Alegre, RS
Latitude: -30.0345365, Longitude: -51.2290198

106
Endereço: Rua Marechal Floriano Peixoto, 333 – Centro. 90020-061 - Porto Alegre, RS
Latitude: -30.03185895, Longitude: -51.227178004983216

105
Endereço: Rua da Repúlblica, 500. - Porto Alegre, RS
Latitude: None, Longitude: None

104
Endereço: Av Aureliano Figueredo

43
Endereço: Rua Múcio Teixeira, 680 - Menino Deus. 90150-090 - Porto Alegre, RS
Latitude: -30.0512052, Longitude: -51.22376797776364

42
Endereço: Rua Otto Niemeyer, 601 – Tristeza. 91910-001 - Porto Alegre, RS
Latitude: None, Longitude: None

41
Endereço: R Vicente da Fontoura, 1135 - Santana - Bairro: - CEP: 90640-001 - Porto Alegre, RS
Latitude: None, Longitude: None

40
Endereço: R Gomes de Freitas, 73 - Jardim Itu - Bairro: - CEP: 91380-000 - Porto Alegre, RS
Latitude: None, Longitude: None

39
Endereço: R Doutor Romaguera da Cunha Correa, 60 - Bairro: - CEP: 91240-070 - Porto Alegre, RS
Latitude: None, Longitude: None

38
Endereço: Av. Cristóvão Colombo, 545 - Prédio 6 Shopping Total. 90560-003 - Porto Alegre, RS
Latitude: None, Longitude: None

37
Endereço: Av. Plínio Brasil Milano, 1000 – Higienópolis. 90520-002 - Porto Alegre, RS
Latitude: -30.0202516, Longitude: -51.1812104

36
Endereço: Rua Lima e Silva, 606 - Cidade Baixa. 90050-102 - Porto Alegre, RS
Latitude: -30.0382877

In [7]:
# Exclui registros que vieram sem coordenadas
tabela_supermercados_com_info = tabela_supermercados[tabela_supermercados.latitude.isna() == False]

## Verificando se uma coordenada está dentro da região do Multipolígono

In [8]:
# Função para processar as coordenadas do KML e criar polígonos
def process_coordinates(coordinate_string):
    coords = coordinate_string.replace('\n', '').split(' ')
    coords = [tuple(map(float, coord.split(',')[:2])) for coord in coords if coord]
    return Polygon(coords)

In [9]:
# Caminho para o arquivo KML
kml_path = 'regioes_afetadas_rs.kml'

# Abrir e ler o arquivo KML
with open(kml_path, 'r') as file:
    root = parser.parse(file).getroot()

# Lista para armazenar todos os polígonos, já que o arquivo no traz um multipolígono (um conjunto de polígonos)
polygons = []

# Iterar sobre todos os Placemarks
namespace = "{http://www.opengis.net/kml/2.2}"

for placemark in root.Document.Folder.findall('.//'+namespace+'Placemark'):
    for polygon in placemark.findall('.//'+namespace+'Polygon'):
        for outer_boundary in polygon.findall('.//'+namespace+'outerBoundaryIs'):
            for linear_ring in outer_boundary.findall('.//'+namespace+'LinearRing'):
                coord_text = linear_ring.coordinates.text
                poly = process_coordinates(coord_text)
                polygons.append(poly)

In [10]:
# Validando se as coordenadas de uma lista se encontram dentro da área de um multipolígono

i = 0
for index, row in tabela_supermercados_com_info.iterrows():
    i += 1
    print(i)
    
    latitude = float(row["latitude"])
    longitude = float(row["longitude"])
    point = Point(longitude, latitude)

    # Verificar se o ponto está em algum polígono
    is_inside = any(poly.contains(point) for poly in polygons)
    
    # Encontra a distância do polígono mais próximo da área
    min_distance = 1000000
    for poly in polygons:
        distance = point.distance(poly)
        if distance < min_distance:
            min_distance = distance         

    # Classifica os pontos conferindo se está dentro, próximo ou fora da região do mapa
    if is_inside:
        print("A coordenada está dentro de pelo menos um polígono.")
        tabela_supermercados_com_info.at[index, "afetado"] = "Região Afetada"
    elif min_distance <= 0.005:  
        print("O ponto está a menos de 500 metros do MultiPolígono.")
        tabela_supermercados_com_info.at[index, "afetado"] = "Região Próxima"
    else:
        print("O ponto não está dentro nem próximo ao MultiPolígono.")
        tabela_supermercados_com_info.at[index, "afetado"] = "Região Segura"

1
O ponto está a menos de 500 metros do MultiPolígono.
2
A coordenada está dentro de pelo menos um polígono.
3
A coordenada está dentro de pelo menos um polígono.
4
O ponto está a menos de 500 metros do MultiPolígono.
5
O ponto está a menos de 500 metros do MultiPolígono.
6
A coordenada está dentro de pelo menos um polígono.
7
A coordenada está dentro de pelo menos um polígono.
8
O ponto não está dentro nem próximo ao MultiPolígono.
9
O ponto está a menos de 500 metros do MultiPolígono.
10
O ponto não está dentro nem próximo ao MultiPolígono.
11
O ponto não está dentro nem próximo ao MultiPolígono.
12
A coordenada está dentro de pelo menos um polígono.
13
A coordenada está dentro de pelo menos um polígono.
14
A coordenada está dentro de pelo menos um polígono.
15
O ponto não está dentro nem próximo ao MultiPolígono.
16
O ponto está a menos de 500 metros do MultiPolígono.
17
O ponto não está dentro nem próximo ao MultiPolígono.
18
O ponto não está dentro nem próximo ao MultiPolígono.
19

In [11]:
tabela_supermercados_com_info.head()

,lista_enderecos,latitude,longitude,afetado
0,Avenida Plínio Brasil Milano 2343. - Porto Ale...,-30.013230,-51.170902,Região Próxima
1,"Av. A. J. Renner, 1603. - Porto Alegre, RS",-29.982955,-51.192105,Região Afetada
2,"Avenida Diario de Noticias, 500. 90810-080 - P...",-30.083234,-51.244600,Região Afetada
5,"Rua Fernando Machado, 860 – Centro. 90010-320 ...",-30.034537,-51.229020,Região Próxima
6,"Rua Marechal Floriano Peixoto, 333 – Centro. 9...",-30.031859,-51.227178,Região Próxima


In [12]:
# Salva o arquivo em formato .xlsx
tabela_supermercados_com_info.to_excel("supermercados_poa.xlsx", index = False)